In [ ]:
import os
import torch
import torchvision
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, TQDMProgressBar
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics import Accuracy

torch.set_float32_matmul_precision('medium') # if you can, disable if fails
print('PyTorch version:', torch.__version__)
print('Lightning version:', L.__version__)
print('CUDA available:', torch.cuda.is_available())
L.seed_everything(42)

/home/davidkimgba/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/davidkimgba/.local/lib/python3.10/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  _C._set_float32_matmul_precision(precision)
Seed set to 42


PyTorch version: 2.9.0+cu128
Lightning version: 2.6.1
CUDA available: True


42

In [ ]:
# Config variables
DATA_DIR   = '../data'        # where CIFAR-10 will be downloaded
OUTPUT_DIR = '../checkpoints' # model checkpoints
LOG_DIR    = '../logs'        # tensorBoard logs

IMG_SIZE    = 224
BATCH_SIZE  = 32
NUM_CLASSES = 10
EPOCHS      = 50
LR          = 3e-4
WEIGHT_DECAY = 1e-4
WORKERS     = 2

USE_SUBSET      = True
SUBSET_FRACTION = 0.2 if USE_SUBSET else 1.0
SUBSET_SEED     = 42

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

In [ ]:
from data_preparation import get_cifar10_loaders

train_loader, val_loader, class_names = get_cifar10_loaders(
    data_dir=DATA_DIR,
    img_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    workers=WORKERS,
    subset_fraction=SUBSET_FRACTION,
    subset_seed=SUBSET_SEED,
)

assert len(class_names) == NUM_CLASSES, f'Expected {NUM_CLASSES} classes, got {len(class_names)}'
print('Classes:', class_names)
print(f'Train samples: {len(train_loader.dataset)}')
print(f'Val samples:   {len(val_loader.dataset)}')

In [ ]:
#model definition (and funcs for logs and lr scheduler)
class BiomeClassifier(L.LightningModule):
    def __init__(self, num_classes, lr, weight_decay, max_epochs):
        super().__init__()
        self.save_hyperparameters()
        self.model = torchvision.models.efficientnet_v2_s(weights=None, num_classes=num_classes)
        self.criterion = torch.nn.CrossEntropyLoss()
        self.val_acc = Accuracy(task='multiclass', num_classes=num_classes)

    def forward(self, x):
        return self.model(x)

    def _shared_step(self, batch):
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)
        return loss, logits, labels

    def training_step(self, batch, batch_idx):
        loss, _, _ = self._shared_step(batch)
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, logits, labels = self._shared_step(batch)
        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_accuracy', self.val_acc(logits, labels), prog_bar=True, on_step=False, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.hparams.max_epochs, eta_min=1e-6)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler}

In [5]:
# training cell
# use tensorboard --logdir=logs during run to view training progress
logger = TensorBoardLogger(save_dir=LOG_DIR, name='e-netv2-cifar10')

callbacks = [
    ModelCheckpoint(dirpath=OUTPUT_DIR, filename='best_efficientnet_v2_s',
                    monitor='val_accuracy', mode='max', save_top_k=1),
    EarlyStopping(monitor='val_accuracy', patience=5, mode='max'),
    TQDMProgressBar(refresh_rate=10),
]

model   = BiomeClassifier(num_classes=NUM_CLASSES, lr=LR, weight_decay=WEIGHT_DECAY, max_epochs=EPOCHS)
trainer = L.Trainer(max_epochs=EPOCHS, callbacks=callbacks, logger=logger)
trainer.fit(model, train_loader, val_loader)
print(f'TensorBoard logs: {logger.log_dir}')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.
2026-02-23 01:20:21.006620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/davidkimgba/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /mnt/c/Users/David Kim/Documents/Programming/Terraria-Biome-Detection/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name      ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ model     │ EfficientNet       │ 20.2 M │ train │     0 │
│ 1 │ criterion │ CrossEntropyLoss   │      0 │ train │     0 │
│ 2 │ val_acc   │ MulticlassAccuracy │      0 │ train │     0 │
└───┴───────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 20.2 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 20.2 M                                                                                               
Total estimated model params size (MB): 80                                                                         
Modules in train mode: 716                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.


TensorBoard logs: ../logs/e-netv2-cifar10/version_1


In [8]:
# display tensorboard logs
%load_ext tensorboard
%tensorboard --logdir ../logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 588665), started 0:21:03 ago. (Use '!kill 588665' to kill it.)

In [9]:
#eval and save
trainer.validate(model, val_loader)

final_path = os.path.join(OUTPUT_DIR, 'final_efficientnet_v2_s.pth')
torch.save(model.model.state_dict(), final_path)
print(f'Saved model to {final_path}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.8289999961853027     │
│         val_loss          │    0.9179585576057434     │
└───────────────────────────┴───────────────────────────┘

Saved model to ../checkpoints/final_efficientnet_v2_s.pth
